# Course Completion Prediction

**Latar Belakang**

Seiring dengan meningkatnya popularitas pendidikan online, institusi pendidikan dan platform e-learning mencari cara untuk meningkatkan tingkat penyelesaian kursus. Banyak peserta kursus online mengalami kesulitan untuk menyelesaikan kursus yang mereka ikuti, yang dapat disebabkan oleh berbagai faktor seperti motivasi, waktu, dan kesulitan materi. Mengetahui faktor-faktor yang mempengaruhi penyelesaian kursus dan dapat memprediksi peserta mana yang kemungkinan besar tidak akan menyelesaikan kursus sangat penting untuk merancang intervensi yang efektif.

**Tujuan Proyek**

Proyek ini bertujuan untuk membangun model machine learning yang dapat memprediksi apakah seorang peserta akan menyelesaikan kursus atau tidak berdasarkan berbagai fitur yang relevan. Dengan prediksi ini, platform e-learning dapat mengambil langkah proaktif untuk meningkatkan tingkat penyelesaian kursus, seperti memberikan dukungan tambahan kepada peserta yang berisiko.

**Business Understanding**

1. **Signifikansi Bisnis**

    Tingkat penyelesaian kursus yang rendah dapat mempengaruhi reputasi dan keberlanjutan platform e-learning. Dengan meningkatkan tingkat penyelesaian, platform dapat meningkatkan kepuasan dan retensi peserta, yang pada gilirannya dapat meningkatkan jumlah pendaftaran kursus dan pendapatan. Selain itu, peserta yang menyelesaikan kursus lebih cenderung merekomendasikan platform kepada orang lain, menciptakan efek word-of-mouth yang positif.

2. **Pemangku Kepentingan**

    Pemangku kepentingan utama dalam proyek ini meliputi:
    - Manajemen platform e-learning
    - Tim pengembangan produk
    - Instruktur dan pembuat konten kursus
    - Peserta kursus

**Data**

Dataset yang digunakan dalam proyek ini diambil dari Kaggle, dengan judul "Predict Online Course Engagement Dataset" yang dapat diakses di [sini](https://www.kaggle.com/datasets/rabieelkharoua/predict-online-course-engagement-dataset). Pada dataset tersebut terdapat 9000 baris dan 9 kolom, dengan kolom sebagai berikut:
- `UserID`: Identifikasi unik untuk setiap pengguna.
- `CourseCategory`: Kategori kursus yang diambil.
- `TimeSpentOnCourse`: Waktu yang dihabiskan peserta pada kursus.
- `NumberOfVideosWatched`: Jumlah video yang ditonton oleh peserta.
- `NumberOfQuizzesTaken`: Jumlah kuis yang diambil oleh peserta.
- `QuizScores`: Skor yang diperoleh peserta dari kuis.
- `CompletionRate`: Tingkat penyelesaian materi kursus.
- `DeviceType`: Jenis perangkat yang digunakan peserta untuk mengakses kursus.
- `CourseCompletion`: Apakah peserta menyelesaikan kursus atau tidak (label target).


## Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import json

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
)

In [5]:
RAW_DATA_PATH = "../data/raw/online_course_engagement_data.csv"

def load_course_completion(path):
    return pd.read_csv(path)

# Load the data 
df = load_course_completion(RAW_DATA_PATH)

In [9]:
df.head()

,UserID,CourseCategory,TimeSpentOnCourse,NumberOfVideosWatched,NumberOfQuizzesTaken,QuizScores,CompletionRate,DeviceType,CourseCompletion
0,5618,Health,29.979719,17,3,50.365656,20.860773,1,0
1,4326,Arts,27.802640,1,5,62.615970,65.632415,1,0
2,5849,Arts,86.820485,14,2,78.458962,63.812007,1,1
3,4992,Science,35.038427,17,10,59.198853,95.433162,0,1
4,3866,Programming,92.490647,16,0,98.428285,18.102478,0,0


## Exploratory Data Analysis (EDA)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   UserID                 9000 non-null   int64  
 1   CourseCategory         9000 non-null   object 
 2   TimeSpentOnCourse      9000 non-null   float64
 3   NumberOfVideosWatched  9000 non-null   int64  
 4   NumberOfQuizzesTaken   9000 non-null   int64  
 5   QuizScores             9000 non-null   float64
 6   CompletionRate         9000 non-null   float64
 7   DeviceType             9000 non-null   int64  
 8   CourseCompletion       9000 non-null   int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 632.9+ KB


- Terdapat 9000 baris dalam dataset dengan 9 kolom/fitur
- Terdapat 1 fitur dengan tipe data object yaitu `CourseCategory`
- Terdapat 5 fitur dengan tipe data int (bilangan bulat) yaitu `UserID`, `TimeSpentOnCourse`, `NumberOfVideosWatched`, `NumberOfQuizzesTaken`, `DeviceType`, dan `CourseCompletion` (fitur target)
- Terdapat 3 fitur dengan tipe data float yaitu `TimeSpentOnCourse`, `QuizScores`, dan `CompletionRate`

In [12]:
df.describe()

,UserID,TimeSpentOnCourse,NumberOfVideosWatched,NumberOfQuizzesTaken,QuizScores,CompletionRate,DeviceType,CourseCompletion
count,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,4498.894556,50.163822,10.024667,5.090556,74.706028,50.340146,0.500667,0.396444
std,2596.849433,28.491750,6.029878,3.157762,14.378383,28.950977,0.500027,0.489186
min,1.000000,1.005230,0.000000,0.000000,50.005119,0.009327,0.000000,0.000000
25%,2251.750000,25.440548,5.000000,2.000000,62.283451,25.653614,0.000000,0.000000
50%,4483.500000,49.818417,10.000000,5.000000,74.743294,50.264124,1.000000,0.000000
75%,6751.250000,75.069924,15.000000,8.000000,87.022663,75.572493,1.000000,1.000000
max,9000.000000,99.992558,20.000000,10.000000,99.994984,99.979711,1.000000,1.000000
